# Simple Substitution Cipher

The affine cipher has about a thousand possible keys but that computers can still brute-force through all of them easily. We need a cipher that has so many possible keys that no computer can brute-force through them all.

The simple substitution cipher is one such cipher that is effectively invulnerable to a brute-force attack because it has an enormous number of possible keys. Even if your computer could try a trillion keys every second, it would still take 12 million years for it to try every one

To implement the simple substitution cipher, we choose a random letter to encrypt each letter of the alphabet, using each letter only once. The key for the simple substitution cipher is always a string of 26 letters of the alphabet in random order. There are 403,291,461,126,605,635,584,000,000 different possible key orderings for the simple substitution cipher. That’s a lot of keys! More important, this number is so large that it’s impossible to brute-force.

In [1]:
LETTERS = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [2]:
def keyIsValid(key):
    keyList = list(key)
    lettersList = list(LETTERS)
    keyList.sort()
    lettersList.sort()

    return keyList == lettersList


def encryptSimpleSubstitutionCipher(key, message):
    return translateSimpleSubstitutionCipher(key, message, 'encrypt')


def decryptSimpleSubstitutionCipher(key, message):
    return translateSimpleSubstitutionCipher(key, message, 'decrypt')


def translateSimpleSubstitutionCipher(key, message, mode):
    translated = ''
    charsA = LETTERS
    charsB = key
    if mode == 'decrypt':
        charsA, charsB = charsB, charsA

    for symbol in message:
        if symbol.upper() in charsA:
            symIndex = charsA.find(symbol.upper())
            if symbol.isupper():
                translated += charsB[symIndex].upper()
            else:
                translated += charsB[symIndex].lower()
        else:
            translated += symbol

    return translated


def getRandomKey():
    key = list(LETTERS)
    random.shuffle(key)
    return ''.join(key)

In [3]:
encryptSimpleSubstitutionCipher('LFWOAYUISVKMNXPBDCRJTQEGHZ', 'Hello, World')

'Iammp, Epcmo'

In [4]:
decryptSimpleSubstitutionCipher('LFWOAYUISVKMNXPBDCRJTQEGHZ', 'Iammp, Epcmo')

'Hello, World'

# Hacking Simple Substitution Cipher

The simple substitution cipher is impossible to crack using brute force because it has too many possible keys. To hack the simple substitution cipher, we need to create a more sophisticated program that uses dictionary values to map the potential decryption letters of a ciphertext.

In brute-force attacks, we try each possible key to check whether it can decrypt the ciphertext. If the key is correct, the decryption results in readable English. But by analyzing the ciphertext first, we can reduce the number of possible keys to try and maybe even find a full or partial key.

Let’s assume the original plaintext consists mostly of words in an English dictionary file, like the one we used in Chapter 11. Although a ciphertext won’t be made of real English words, it will still contain groups of letters broken up by spaces, just like words in regular sentences.

Because each plaintext letter can encrypt to only one cipherletter, and we’re not encrypting spaces in this version of the cipher, the plaintext and ciphertext will share the same word patterns.

For example, if we had the plaintext MISSISSIPPI SPILL, the corresponding ciphertext might be RJBBJBBJXXJ BXJHH. The number of letters in the first word of the plaintext and the first cipherword are the same. The same is true for the second plaintext word and the second cipher­word. The plaintext and ciphertext share the same pattern of letters and spaces. Also notice that letters that repeat in the plaintext repeat the same number of times and in the same places as the ciphertext.

We could therefore assume that a cipherword corresponds to a word in the English dictionary file and that their word patterns would match. Then, if we can find which word in the dictionary the cipherword decrypts to, we can figure out the decryption of each cipherletter in that word. And if we figure out enough cipherletter decryptions using this technique, we may be able to decrypt the entire message.

## Finding Word Patterns
Let’s examine the word pattern of the cipherword HGHHU. You can see that the cipherword has certain characteristics, which the original plaintext word must share. Both words must have the following in common.

* They should be five letters long.

* The first, third, and fourth letters should be the same.

* They should have exactly three different letters; the first, second, and fifth letters should all be different.

Let’s think of words in the English language that fit this pattern. Puppy is one such word, which is five letters long (P, U, P, P, Y) and uses three different letters (P, U, Y) arranged in that same pattern (P for the first, third, and fourth letter; U for the second letter; and Y for the fifth letter). Mommy, bobby, lulls, and nanny fit the pattern, too. These words, along with any other word in the English dictionary file that matches the criteria, are all possible decryptions of HGHHU.

To represent a word pattern in a way the program can understand, we’ll make each pattern into a set of numbers separated by periods that indicates the pattern of letters.

Creating word patterns is easy: the first letter gets the number 0, and the first occurrence of each different letter thereafter gets the next number. For example, the word pattern for cat is 0.1.2, and the word pattern for classification is 0.1.2.3.3.4.5.4.0.2.6.4.7.8.

In simple substitution ciphers, no matter which key is used to encrypt, a plaintext word and its cipherword always have the same word pattern. The word pattern for the cipherword HGHHU is 0.1.0.0.2, which means the word pattern of the plaintext corresponding to HGHHU is also 0.1.0.0.2.

## Finding Potential Decryption Letters
To decrypt HGHHU, we need to find all the words in an English dictionary file whose word pattern is also 0.1.0.0.2. In this book, we’ll call the plaintext words that have the same word pattern as the cipherword the candidates for that cipherword. Here is a list of candidates for HGHHU:

* puppy
* mommy
* bobby
* lulls
* nanny

Using word patterns, we can guess which plaintext letters cipherletters might decrypt to, which we’ll call the cipherletter’s potential decryption letters. To crack a message encrypted with the simple substitution cipher, we need to find all the potential decryption letters of each word in the message and determine the actual decryption letters through the process of elimination

* H has the potential decryption letters P, M, B, L, and N.
* G has the potential decryption letters U, O, and A.
* U has the potential decryption letters Y and S.

All of the other cipherletters besides H, G, and U have no potential decryption letters in this example.

A cipherletter mapping shows all the letters of the alphabet and their potential decryption letters. As we start to gather encrypted messages, we’ll find potential decryption letters for every letter in the alphabet, but because only the cipherletters H, G, and U were part of our example ciphertext, we don’t have the potential decryption letters of other cipherletters.

We’ll use a dictionary value to represent cipherletter mappings. This dictionary has 26 key-value pairs, one key for each letter of the alphabet and a list of potential decryption letters for each letter. 

If we can reduce the number of potential decryption letters for a cipherletter to just one letter by cross-referencing cipherletter mappings of other encrypted words, we can find what that cipherletter decrypts to. Even if we can’t solve all 26 cipherletters, we might be able to hack most of the cipherletter mappings to decrypt most of the ciphertext.

## Overview of the Hacking Process
Hacking the simple substitution cipher is pretty easy using word patterns. We can summarize the major steps of the hacking process as follows:

* Find the word pattern for each cipherword in the ciphertext.
* Find the English word candidates that each cipherword could decrypt to.
* Create a dictionary showing potential decryption letters for each cipherletter to act as the cipherletter mapping for each cipherword.
* Combine the cipherletter mappings into a single mapping, which we’ll call an intersected mapping.
* Remove any solved cipherletters from the combined mapping.
* Decrypt the ciphertext with the solved cipherletters.

The more cipherwords in a ciphertext, the more likely it is for the mappings to overlap with one another and the fewer the potential decryption letters for each cipherletter. This means that in the simple substitution cipher, the longer the ciphertext message, the easier it is to hack.

In [5]:
from cracking_codes.utils import wordpatterns, makewordpatterns

In [6]:
import re, copy

In [7]:
nonLettersOrSpacePattern = re.compile('[^A-Z\s]')

In [8]:
def getBlankCipherletterMapping():
    return {'A': [], 'B': [], 'C': [], 'D': [], 'E': [], 'F': [], 'G': [],
       'H': [], 'I': [], 'J': [], 'K': [], 'L': [], 'M': [], 'N': [],
       'O': [], 'P': [], 'Q': [], 'R': [], 'S': [], 'T': [], 'U': [],
       'V': [], 'W': [], 'X': [], 'Y': [], 'Z': []}


def addLettersToMapping(letterMapping, cipherword, candidate):
    for i in range(len(cipherword)):
        if candidate[i] not in letterMapping[cipherword[i]]:
            letterMapping[cipherword[i]].append(candidate[i])


def intersectMappings(mapA, mapB):
    intersectedMapping = getBlankCipherletterMapping()
    for letter in LETTERS:
        if mapA[letter] == []:
            intersectedMapping[letter] = copy.deepcopy(mapB[letter])
        elif mapB[letter] == []:
            intersectedMapping[letter] = copy.deepcopy(mapA[letter])
        else:
            for mappedLetter in mapA[letter]:
                if mappedLetter in mapB[letter]:
                    intersectedMapping[letter].append(mappedLetter)

    return intersectedMapping


def removeSolvedLettersFromMapping(letterMapping):
    loopAgain = True
    while loopAgain:
        loopAgain = False

        solvedLetters = []
        for cipherletter in LETTERS:
            if len(letterMapping[cipherletter]) == 1:
                solvedLetters.append(letterMapping[cipherletter][0])

        for cipherletter in LETTERS:
            for s in solvedLetters:
                if len(letterMapping[cipherletter]) != 1 and s in letterMapping[cipherletter]:
                    letterMapping[cipherletter].remove(s)
                    if len(letterMapping[cipherletter]) == 1:
                        loopAgain = True
    return letterMapping


def hackSimpleSubstitutionCipher(message):
    intersectedMap = getBlankCipherletterMapping()
    cipherwordList = nonLettersOrSpacePattern.sub('', message.upper()).split()
    for cipherword in cipherwordList:
        candidateMap = getBlankCipherletterMapping()

        wordPattern = makewordpatterns.getWordPattern(cipherword)
        if wordPattern not in wordpatterns.allPatterns:
            continue 
        for candidate in wordpatterns.allPatterns[wordPattern]:
            addLettersToMapping(candidateMap, cipherword, candidate)

        intersectedMap = intersectMappings(intersectedMap, candidateMap)

    return removeSolvedLettersFromMapping(intersectedMap)


def decryptWithCipherletterMapping(ciphertext, letterMapping):

    key = ['x'] * len(LETTERS)
    for cipherletter in LETTERS:
        if len(letterMapping[cipherletter]) == 1:
            keyIndex = LETTERS.find(letterMapping[cipherletter][0])
            key[keyIndex] = cipherletter
        else:
            ciphertext = ciphertext.replace(cipherletter.lower(), '_')
            ciphertext = ciphertext.replace(cipherletter.upper(), '_')
    key = ''.join(key)

    return decryptSimpleSubstitutionCipher(key, ciphertext)

In [9]:
message = """Sy l nlx sr pyyacao l ylwj eiswi upar lulsxrj isr sxrjsxwjr, ia esmm rwctjsxsza sj wmpramh, lxo txmarr jia
aqsoaxwa sr pqaceiamnsxu, ia esmm caytra jp famsaqa sj. Sy, px jia pjiac ilxo, ia sr pyyacao rpnajisxu eiswi lyypcor l calrpx
ypc lwjsxu sx lwwpcolxwa jp isr sxrjsxwjr, ia esmm lwwabj sj aqax px jia rmsuijarj aqsoaxwa. Jia pcsusx py nhjir sr agbmlsxao
sx jisr elh. -Facjclxo Ctrramm"""

In [10]:
letterMapping = hackSimpleSubstitutionCipher(message)

In [11]:
decryptWithCipherletterMapping(message, letterMapping)

'If a man is offered a fact which goes against his instincts, he will scrutinize it closel_, and unless the\nevidence is overwhelming, he will refuse to _elieve it. If, on the other hand, he is offered something which affords a reason\nfor acting in accordance to his instincts, he will acce_t it even on the slightest evidence. The origin of m_ths is e__lained\nin this wa_. -_ertrand Russell'